In [1]:
import os
import json
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch.nn as nn

In [24]:
# Load the Yelp businesses and reviews datasets
businesses_df = pd.read_feather(
    os.path.join('data', 'yelp', 'yelp_academic_dataset_business.feather'),
    columns=['business_id', 'categories', 'review_count', 'stars']
    )
print('buinesses_df loaded')

reviews_df = pd.read_feather(
    os.path.join('data', 'yelp', 'yelp_academic_dataset_review.feather'),
    columns=['review_id', 'user_id', 'business_id', 'stars'])
print('reviews_df loaded')

# Filter businesses by category (e.g., restaurants)
print('Filter businesses by category (e.g., restaurants)')
businesses_df = businesses_df.where(
    businesses_df['categories'].str.contains('Restaurants', na=False)
    ).dropna()

# Extract unique restaurant IDs and create a dictionary mapping them to indices
print('Extract unique restaurant IDs and create a dictionary mapping them to indices')
restaurant_ids = businesses_df['business_id'].unique().tolist()
num_restaurants = len(restaurant_ids)

# Encode the restaurant IDs 
restaurant_indices = dict(zip(restaurant_ids, range(num_restaurants)))
restaurant_indices_inv = {v: k for k, v in restaurant_indices.items()}

buinesses_df loaded
reviews_df loaded
Filter businesses by category (e.g., restaurants)
Extract unique restaurant IDs and create a dictionary mapping them to indices


In [3]:
# reviews_df = reviews_df[['review_id', 'user_id', 'business_id', 'stars', 'text']]
# top20k = businesses_df['business_id'].value_counts()[:20000].index.tolist()

In [4]:
# reviews_subdf = reviews_df[reviews_df['user_id'].isin(top20k)]
# businesses_subdf = businesses_df[businesses_df['business_id'].isin(top20k)]

In [25]:
# Convert categories to string type
businesses_df['categories'] = businesses_df['categories'].astype(str)

# Encode the categorical features using label encoding
label_encoder = LabelEncoder()
businesses_df['categories'] = label_encoder.fit_transform(businesses_df['categories'])


In [28]:
# Convert the node_features array to a suitable data type
node_features = businesses_df[['categories', 'review_count', 'stars']].values.astype(np.float32)
node_features = torch.FloatTensor(node_features)
reviews_subdf = reviews_df.merge(businesses_df[['business_id']], on='business_id')
restaurant_ratings = reviews_df.groupby(['business_id', 'user_id'])['stars'].mean().reset_index()


In [30]:
reviews_df = reviews_df.merge(businesses_df[['business_id']], on='business_id')
restaurant_ratings = reviews_df.groupby(['business_id', 'user_id'])['stars'].mean().reset_index()


In [32]:
restaurant_ratings = restaurant_ratings.dropna(subset=['business_id', 'user_id'])


In [33]:
restaurant_ratings

,business_id,user_id,stars
typing.Literal[<no_default>],,,
0,---kPU91CF4Lq2-WlRu9Lw,415SXXbQrl2-VudHa3ImbQ,4.0
1,---kPU91CF4Lq2-WlRu9Lw,5jlO2REcgB6GKFeSsc-OXw,5.0
2,---kPU91CF4Lq2-WlRu9Lw,5r_jlIQvSr7VG5YFbhU6nw,5.0
3,---kPU91CF4Lq2-WlRu9Lw,6SoUQtbIltsun0IIGqWTqA,4.0
4,---kPU91CF4Lq2-WlRu9Lw,7DKv40qZEDctS9G0M6jkgQ,5.0
...,...,...,...
4561437,zzyx5x0Z7xXWWvWnZFuxlQ,76PW6djrso_xuIL_2QZpfA,3.0
4561438,zzyx5x0Z7xXWWvWnZFuxlQ,GMFNrTTflpbBmPITdpsgEg,1.0
4561439,zzyx5x0Z7xXWWvWnZFuxlQ,NGwwlCPEfZRWHxkQ7BD7WA,4.0


In [31]:
user_ratings_df = restaurant_ratings.pivot(index='business_id', columns='user_id', values='stars').fillna(0)


c:\agh\MAGISTERKA\restaurant-recommender\venv\lib\site-packages\pandas\core\reshape\reshape.py:125: RuntimeWarning: overflow encountered in long_scalars
  num_cells = num_rows * num_columns


ValueError: negative dimensions are not allowed

In [22]:


# Create a sparse tensor representing the restaurant ratings

user_ratings_df = user_ratings_df.rename(index=restaurant_indices_inv)
user_ratings = torch.FloatTensor(user_ratings_df.values)

# Create adjacency matrix and node features
adjacency_matrix = torch.zeros((num_restaurants, num_restaurants))
for _, row in businesses_df.iterrows():
    i = restaurant_indices[row['business_id']]
    j = restaurant_indices[row['business_id']]
    adjacency_matrix[i, j] = 1
    adjacency_matrix[j, i] = 1

node_features = businesses_df[['categories', 'review_count', 'stars']].values
node_features = torch.FloatTensor(node_features)

c:\agh\MAGISTERKA\restaurant-recommender\venv\lib\site-packages\pandas\core\reshape\reshape.py:125: RuntimeWarning: overflow encountered in long_scalars
  num_cells = num_rows * num_columns


ValueError: negative dimensions are not allowed

In [8]:
# Define the YelpGNN model
class YelpGNN(nn.Module):
    def __init__(self, num_features, hidden_channels):
        super(YelpGNN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.fc(x)
        return x




In [9]:
node_features.shape

torch.Size([20000, 3])

In [14]:
adjacency_matrix.nonzero().t().long().shape

torch.Size([2, 20000])

In [15]:
# Create the YelpGNN model instance
hidden_channels = 64
model = YelpGNN(node_features.shape[1], hidden_channels)
data = Data(x=node_features, edge_index=adjacency_matrix.nonzero().t().long())

# Set up the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

In [16]:
model.train()
optimizer.zero_grad()
output = model(data.x, data.edge_index)

In [17]:
user_ratings.shape

torch.Size([377, 194])

In [18]:
output.shape

torch.Size([20000, 1])

In [18]:




# Train the model
for epoch in range(1):
    model.train()
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = criterion(output, user_ratings)

    # Add regularization terms to the loss function to encourage diversity in the recommendations
    user_embeddings = model.conv2(model.conv1(data.x, data.edge_index))
    user_embeddings = user_embeddings[:num_restaurants] / torch.norm(user_embeddings[:num_restaurants], dim=1, keepdim=True)
    item_embeddings = data.x[:num_restaurants]
    item_embeddings = item_embeddings / torch.norm(item_embeddings, dim=1, keepdim=True)
    diversity_loss = -torch.mean(torch.matmul(user_embeddings, item_embeddings.t()))
    loss += 0.01 * diversity_loss

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print("Epoch {:03d}, Loss: {:.4f}".format(epoch, loss.item()))

c:\agh\MAGISTERKA\restaurant-recommender\venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([377, 194])) that is different to the input size (torch.Size([20000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (20000) must match the size of tensor b (377) at non-singleton dimension 0